# Working code

In [1]:
import pandas as pd
from datetime import timedelta
pd.set_option('display.max_columns', None)

# Load the data
sentinel_data_path = 'Sentinel2_Data.csv'
ground_data_path = 'Ground_Data.csv'

# Read the CSV files
sentinel_df = pd.read_csv(sentinel_data_path, parse_dates=['timestamp'])
ground_df = pd.read_csv(ground_data_path, parse_dates=['Timestamp (UTC+12:00)'], low_memory=False)

# Rename timestamp columns
sentinel_df.rename(columns={'timestamp': 'timestamp_sentinel2'}, inplace=True)
ground_df.rename(columns={'Timestamp (UTC+12:00)': 'timestamp_ground'}, inplace=True)

# Convert columns to appropriate data types
numeric_cols = [
    'Discharge_(m^3/s)', 'Lake_Height_(m)', 'PercentFull_Active_Lake_Storage_(%)',
    'Snow_Volume_Opuha_Catchment_(mm)', 'Turbidity_Buoy_(NTU)', 'Turbidity_Platform_(NTU)',
    'Water_Temp_Buoy_(degC)', 'Water_Temp_Platform_(degC)', 'WDir(Deg)', 'WSpd(m/s)',
    'GustDir(Deg)', 'GustSpd(m/s)', 'WindRun(Km)', 'Rain(mm)', 'Tdry(C)', 'TWet(C)',
    'RH(%)', 'Tmax(C)', 'Tmin(C)', 'Pmsl(hPa)', 'Pstn(hPa)'
]

for col in numeric_cols:
    ground_df[col] = pd.to_numeric(ground_df[col], errors='coerce')

# Separate ground data based on sampling intervals
three_hour_avg_cols = ['Discharge_(m^3/s)']
fifteen_min_avg_cols = [
    'Water_Temp_Buoy_(degC)', 'Water_Temp_Platform_(degC)',
    'Turbidity_Buoy_(NTU)', 'Turbidity_Platform_(NTU)'
]
daily_avg_cols = [
    'Lake_Height_(m)', 'PercentFull_Active_Lake_Storage_(%)', 'Snow_Volume_Opuha_Catchment_(mm)',
    'WDir(Deg)', 'WSpd(m/s)', 'GustDir(Deg)', 'GustSpd(m/s)', 'WindRun(Km)', 'Rain(mm)',
    'Tdry(C)', 'TWet(C)', 'RH(%)', 'Tmax(C)', 'Tmin(C)', 'Pmsl(hPa)', 'Pstn(hPa)'
]

# Function to find the closest ground measurement to a given Sentinel-2 timestamp and compute time difference
def find_closest_measurement_time(sentinel_time, ground_df, cols, time_window):
    start_time = sentinel_time - time_window
    end_time = sentinel_time + time_window
    filtered_ground_df = ground_df[(ground_df['timestamp_ground'] >= start_time) & (ground_df['timestamp_ground'] <= end_time)]
    if not filtered_ground_df.empty:
        closest_time = filtered_ground_df['timestamp_ground'].iloc[(filtered_ground_df['timestamp_ground'] - sentinel_time).abs().argsort()[:1]].iloc[0]
        time_diff = abs((closest_time - sentinel_time).total_seconds())  # Time difference in seconds
        closest_rows = filtered_ground_df[filtered_ground_df['timestamp_ground'] == closest_time][cols].iloc[0].to_dict()
        return closest_rows, time_diff
    else:
        return {col: None for col in cols}, None

# Define the time window for finer intervals (e.g., 1 hour)
time_window = timedelta(hours=3)

# Prepare a list to store merged data
merged_data = []

# Process each Sentinel-2 timestamp
for index, row in sentinel_df.iterrows():
    sentinel_time = row['timestamp_sentinel2']
    merged_row = row.to_dict()
    
    # Find closest measurements and time difference for three hour average and fifteen minute interval columns
    closest_three_hour, time_diff_three_hour = find_closest_measurement_time(sentinel_time, ground_df, three_hour_avg_cols, time_window)
    closest_fifteen_min, time_diff_fifteen_min = find_closest_measurement_time(sentinel_time, ground_df, fifteen_min_avg_cols, time_window)
    
    # Update the merged row with these measurements
    merged_row.update(closest_three_hour)
    merged_row.update(closest_fifteen_min)
    
    # Add the time difference to the merged row (use the time difference from the finer interval measurements)
    merged_row['Ground_Measurements_time_diff_(seconds)'] = time_diff_fifteen_min if time_diff_fifteen_min is not None else time_diff_three_hour
    
    merged_data.append(merged_row)

# Convert the merged data to a DataFrame
merged_df = pd.DataFrame(merged_data)

# Create date columns for merging daily averages
ground_df['date'] = ground_df['timestamp_ground'].dt.date
sentinel_df['date'] = sentinel_df['timestamp_sentinel2'].dt.date
merged_df['date'] = pd.to_datetime(merged_df['timestamp_sentinel2']).dt.date

# Calculate daily averages for relevant ground data
daily_avg_df = ground_df.groupby('date')[daily_avg_cols].mean().reset_index()

# Merge the daily averages with the Sentinel-2 data
merged_df = pd.merge(merged_df, daily_avg_df, on='date', how='left')

# Drop the 'date' column as it was only for merging purposes
merged_df.drop(columns=['date'], inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('New_Merged_Data.csv', index=False)

print("Merging completed. The merged dataset is saved as 'New_Merged_Data.csv'")


Merging completed. The merged dataset is saved as 'New_Merged_Data.csv'


In [2]:

# Load the merged data
merged_data_path = 'New_Merged_Data.csv'
merged_df = pd.read_csv(merged_data_path)

# List of band prefixes to aggregate
bands = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']

# Compute averages for each band
for band in bands:
    band_cols = [col for col in merged_df.columns if f'_{band}' in col]
    merged_df[f'{band}_AVG'] = merged_df[band_cols].mean(axis=1)

# Compute averages for cs and cs_CDF
cs_cols = [col for col in merged_df.columns if '_cs' in col and '_cs_cdf' not in col]
cs_cdf_cols = [col for col in merged_df.columns if '_cs_cdf' in col]

merged_df['cs_AVG'] = merged_df[cs_cols].mean(axis=1)
merged_df['cs_cdf_AVG'] = merged_df[cs_cdf_cols].mean(axis=1)

# Save the updated dataframe to a new CSV file
updated_data_path = 'Updated_Merged_Data.csv'
merged_df.to_csv(updated_data_path, index=False)

print("Aggregation completed. The updated dataset is saved as 'Updated_Merged_Data.csv'")


Aggregation completed. The updated dataset is saved as 'Updated_Merged_Data.csv'


In [3]:
merged_df

,timestamp_sentinel2,point1_B2,point10_B2,point11_B2,point12_B2,point13_B2,point14_B2,point15_B2,point16_B2,point17_B2,point18_B2,point19_B2,point2_B2,point20_B2,point21_B2,point22_B2,point23_B2,point24_B2,point25_B2,point26_B2,point27_B2,point28_B2,point29_B2,point3_B2,point30_B2,point31_B2,point32_B2,point33_B2,point34_B2,point35_B2,point36_B2,point37_B2,point38_B2,point39_B2,point4_B2,point40_B2,point41_B2,point42_B2,point43_B2,point44_B2,point45_B2,point46_B2,point47_B2,point48_B2,point49_B2,point5_B2,point50_B2,point51_B2,point52_B2,point53_B2,point54_B2,point55_B2,point56_B2,point57_B2,point58_B2,point59_B2,point6_B2,point60_B2,point61_B2,point62_B2,point63_B2,point64_B2,point65_B2,point66_B2,point7_B2,point8_B2,point9_B2,point1_B3,point10_B3,point11_B3,point12_B3,point13_B3,point14_B3,point15_B3,point16_B3,point17_B3,point18_B3,point19_B3,point2_B3,point20_B3,point21_B3,point22_B3,point23_B3,point24_B3,point25_B3,point26_B3,point27_B3,point28_B3,point29_B3,point3_B3,point30_B3,point31_B3,point32_B3,point33_B3,point34_B3,point35_B3,point36_B3,point37_B3,point38_B3,point39_B3,point4_B3,point40_B3,point41_B3,point42_B3,point43_B3,point44_B3,point45_B3,point46_B3,point47_B3,point48_B3,point49_B3,point5_B3,point50_B3,point51_B3,point52_B3,point53_B3,point54_B3,point55_B3,point56_B3,point57_B3,point58_B3,point59_B3,point6_B3,point60_B3,point61_B3,point62_B3,point63_B3,point64_B3,point65_B3,point66_B3,point7_B3,point8_B3,point9_B3,point1_B4,point10_B4,point11_B4,point12_B4,point13_B4,point14_B4,point15_B4,point16_B4,point17_B4,point18_B4,point19_B4,point2_B4,point20_B4,point21_B4,point22_B4,point23_B4,point24_B4,point25_B4,point26_B4,point27_B4,point28_B4,point29_B4,point3_B4,point30_B4,point31_B4,point32_B4,point33_B4,point34_B4,point35_B4,point36_B4,point37_B4,point38_B4,point39_B4,point4_B4,point40_B4,point41_B4,point42_B4,point43_B4,point44_B4,point45_B4,point46_B4,point47_B4,point48_B4,point49_B4,point5_B4,point50_B4,point51_B4,point52_B4,point53_B4,point54_B4,point55_B4,point56_B4,point57_B4,point58_B4,point59_B4,point6_B4,point60_B4,point61_B4,point62_B4,point63_B4,point64_B4,point65_B4,point66_B4,point7_B4,point8_B4,point9_B4,point1_B8,point10_B8,point11_B8,point12_B8,point13_B8,point14_B8,point15_B8,point16_B8,point17_B8,point18_B8,point19_B8,point2_B8,point20_B8,point21_B8,point22_B8,point23_B8,point24_B8,point25_B8,point26_B8,point27_B8,point28_B8,point29_B8,point3_B8,point30_B8,point31_B8,point32_B8,point33_B8,point34_B8,point35_B8,point36_B8,point37_B8,point38_B8,point39_B8,point4_B8,point40_B8,point41_B8,point42_B8,point43_B8,point44_B8,point45_B8,point46_B8,point47_B8,point48_B8,point49_B8,point5_B8,point50_B8,point51_B8,point52_B8,point53_B8,point54_B8,point55_B8,point56_B8,point57_B8,point58_B8,point59_B8,point6_B8,point60_B8,point61_B8,point62_B8,point63_B8,point64_B8,point65_B8,point66_B8,point7_B8,point8_B8,point9_B8,point1_B8A,point10_B8A,point11_B8A,point12_B8A,point13_B8A,point14_B8A,point15_B8A,point16_B8A,point17_B8A,point18_B8A,point19_B8A,point2_B8A,point20_B8A,point21_B8A,point22_B8A,point23_B8A,point24_B8A,point25_B8A,point26_B8A,point27_B8A,point28_B8A,point29_B8A,point3_B8A,point30_B8A,point31_B8A,point32_B8A,point33_B8A,point34_B8A,point35_B8A,point36_B8A,point37_B8A,point38_B8A,point39_B8A,point4_B8A,point40_B8A,point41_B8A,point42_B8A,point43_B8A,point44_B8A,point45_B8A,point46_B8A,point47_B8A,point48_B8A,point49_B8A,point5_B8A,point50_B8A,point51_B8A,point52_B8A,point53_B8A,point54_B8A,point55_B8A,point56_B8A,point57_B8A,point58_B8A,point59_B8A,point6_B8A,point60_B8A,point61_B8A,point62_B8A,point63_B8A,point64_B8A,point65_B8A,point66_B8A,point7_B8A,point8_B8A,point9_B8A,point1_B11,point10_B11,point11_B11,point12_B11,point13_B11,point14_B11,point15_B11,point16_B11,point17_B11,point18_B11,point19_B11,point2_B11,point20_B11,point21_B11,point22_B11,point23_B11,point24_B11,point25_B11,point26_B11,point27_B11,point28_B11,point29_B11,point3_B11,point30_B11,point31_B11,point32_B11,point33_B11,point34

# unknow issue of not able to merge the discharge column, so we drop it first then mannualy merge it with the discharge table

In [6]:
merged_df=merged_df.drop(columns=['Discharge_(m^3/s)'])

KeyError: "['Discharge_(m^3/s)'] not found in axis"

In [7]:
merged_df

,timestamp_sentinel2,point1_B2,point10_B2,point11_B2,point12_B2,point13_B2,point14_B2,point15_B2,point16_B2,point17_B2,point18_B2,point19_B2,point2_B2,point20_B2,point21_B2,point22_B2,point23_B2,point24_B2,point25_B2,point26_B2,point27_B2,point28_B2,point29_B2,point3_B2,point30_B2,point31_B2,point32_B2,point33_B2,point34_B2,point35_B2,point36_B2,point37_B2,point38_B2,point39_B2,point4_B2,point40_B2,point41_B2,point42_B2,point43_B2,point44_B2,point45_B2,point46_B2,point47_B2,point48_B2,point49_B2,point5_B2,point50_B2,point51_B2,point52_B2,point53_B2,point54_B2,point55_B2,point56_B2,point57_B2,point58_B2,point59_B2,point6_B2,point60_B2,point61_B2,point62_B2,point63_B2,point64_B2,point65_B2,point66_B2,point7_B2,point8_B2,point9_B2,point1_B3,point10_B3,point11_B3,point12_B3,point13_B3,point14_B3,point15_B3,point16_B3,point17_B3,point18_B3,point19_B3,point2_B3,point20_B3,point21_B3,point22_B3,point23_B3,point24_B3,point25_B3,point26_B3,point27_B3,point28_B3,point29_B3,point3_B3,point30_B3,point31_B3,point32_B3,point33_B3,point34_B3,point35_B3,point36_B3,point37_B3,point38_B3,point39_B3,point4_B3,point40_B3,point41_B3,point42_B3,point43_B3,point44_B3,point45_B3,point46_B3,point47_B3,point48_B3,point49_B3,point5_B3,point50_B3,point51_B3,point52_B3,point53_B3,point54_B3,point55_B3,point56_B3,point57_B3,point58_B3,point59_B3,point6_B3,point60_B3,point61_B3,point62_B3,point63_B3,point64_B3,point65_B3,point66_B3,point7_B3,point8_B3,point9_B3,point1_B4,point10_B4,point11_B4,point12_B4,point13_B4,point14_B4,point15_B4,point16_B4,point17_B4,point18_B4,point19_B4,point2_B4,point20_B4,point21_B4,point22_B4,point23_B4,point24_B4,point25_B4,point26_B4,point27_B4,point28_B4,point29_B4,point3_B4,point30_B4,point31_B4,point32_B4,point33_B4,point34_B4,point35_B4,point36_B4,point37_B4,point38_B4,point39_B4,point4_B4,point40_B4,point41_B4,point42_B4,point43_B4,point44_B4,point45_B4,point46_B4,point47_B4,point48_B4,point49_B4,point5_B4,point50_B4,point51_B4,point52_B4,point53_B4,point54_B4,point55_B4,point56_B4,point57_B4,point58_B4,point59_B4,point6_B4,point60_B4,point61_B4,point62_B4,point63_B4,point64_B4,point65_B4,point66_B4,point7_B4,point8_B4,point9_B4,point1_B8,point10_B8,point11_B8,point12_B8,point13_B8,point14_B8,point15_B8,point16_B8,point17_B8,point18_B8,point19_B8,point2_B8,point20_B8,point21_B8,point22_B8,point23_B8,point24_B8,point25_B8,point26_B8,point27_B8,point28_B8,point29_B8,point3_B8,point30_B8,point31_B8,point32_B8,point33_B8,point34_B8,point35_B8,point36_B8,point37_B8,point38_B8,point39_B8,point4_B8,point40_B8,point41_B8,point42_B8,point43_B8,point44_B8,point45_B8,point46_B8,point47_B8,point48_B8,point49_B8,point5_B8,point50_B8,point51_B8,point52_B8,point53_B8,point54_B8,point55_B8,point56_B8,point57_B8,point58_B8,point59_B8,point6_B8,point60_B8,point61_B8,point62_B8,point63_B8,point64_B8,point65_B8,point66_B8,point7_B8,point8_B8,point9_B8,point1_B8A,point10_B8A,point11_B8A,point12_B8A,point13_B8A,point14_B8A,point15_B8A,point16_B8A,point17_B8A,point18_B8A,point19_B8A,point2_B8A,point20_B8A,point21_B8A,point22_B8A,point23_B8A,point24_B8A,point25_B8A,point26_B8A,point27_B8A,point28_B8A,point29_B8A,point3_B8A,point30_B8A,point31_B8A,point32_B8A,point33_B8A,point34_B8A,point35_B8A,point36_B8A,point37_B8A,point38_B8A,point39_B8A,point4_B8A,point40_B8A,point41_B8A,point42_B8A,point43_B8A,point44_B8A,point45_B8A,point46_B8A,point47_B8A,point48_B8A,point49_B8A,point5_B8A,point50_B8A,point51_B8A,point52_B8A,point53_B8A,point54_B8A,point55_B8A,point56_B8A,point57_B8A,point58_B8A,point59_B8A,point6_B8A,point60_B8A,point61_B8A,point62_B8A,point63_B8A,point64_B8A,point65_B8A,point66_B8A,point7_B8A,point8_B8A,point9_B8A,point1_B11,point10_B11,point11_B11,point12_B11,point13_B11,point14_B11,point15_B11,point16_B11,point17_B11,point18_B11,point19_B11,point2_B11,point20_B11,point21_B11,point22_B11,point23_B11,point24_B11,point25_B11,point26_B11,point27_B11,point28_B11,point29_B11,point3_B11,point30_B11,point31_B11,point32_B11,point33_B11,point34

In [9]:
merged_df.to_csv('merged_data_without_discharge.csv', index=False)

# Fix up for the discharge

In [10]:
# Load the data
merged_data_path = 'merged_data_without_discharge.csv'
discharge_data_path = 'Discharge.csv'

# Read the merged dataset
merged_df = pd.read_csv(merged_data_path, parse_dates=['timestamp_sentinel2'])

# Read the discharge data and skip the first row (metadata)
discharge_df = pd.read_csv(discharge_data_path, skiprows=1, parse_dates=['Timestamp (UTC+12:00)'])

# Rename columns for consistency
discharge_df.rename(columns={'Timestamp (UTC+12:00)': 'timestamp_discharge', 'Value (m^3/s)': 'Discharge_(m^3/s)'}, inplace=True)

# Function to find the closest discharge measurement to a given Sentinel-2 timestamp
def find_closest_discharge(sentinel_time, discharge_df):
    time_diffs = (discharge_df['timestamp_discharge'] - sentinel_time).abs()
    closest_idx = time_diffs.idxmin()
    closest_value = discharge_df.at[closest_idx, 'Discharge_(m^3/s)']
    closest_time = discharge_df.at[closest_idx, 'timestamp_discharge']
    time_diff = abs((closest_time - sentinel_time).total_seconds())  # Time difference in seconds
    return closest_value, time_diff

# Prepare a list to store merged data
merged_with_discharge_data = []

# Process each Sentinel-2 timestamp
for index, row in merged_df.iterrows():
    sentinel_time = row['timestamp_sentinel2']
    merged_row = row.to_dict()
    
    # Find the closest discharge measurement
    closest_discharge, discharge_time_diff = find_closest_discharge(sentinel_time, discharge_df)
    merged_row['Discharge_(m^3/s)'] = closest_discharge
    merged_row['Discharge_time_diff_(seconds)'] = discharge_time_diff
    
    merged_with_discharge_data.append(merged_row)

# Convert the merged data to a DataFrame
merged_with_discharge_df = pd.DataFrame(merged_with_discharge_data)

# Save the merged dataframe to a CSV file
merged_with_discharge_df.to_csv('Data_Merged.csv', index=False)

print("Merging completed. The merged dataset with discharge is saved as 'Data_Merged.csv'")

Merging completed. The merged dataset with discharge is saved as 'Data_Merged.csv'


In [11]:
merged_with_discharge_df

,timestamp_sentinel2,point1_B2,point10_B2,point11_B2,point12_B2,point13_B2,point14_B2,point15_B2,point16_B2,point17_B2,point18_B2,point19_B2,point2_B2,point20_B2,point21_B2,point22_B2,point23_B2,point24_B2,point25_B2,point26_B2,point27_B2,point28_B2,point29_B2,point3_B2,point30_B2,point31_B2,point32_B2,point33_B2,point34_B2,point35_B2,point36_B2,point37_B2,point38_B2,point39_B2,point4_B2,point40_B2,point41_B2,point42_B2,point43_B2,point44_B2,point45_B2,point46_B2,point47_B2,point48_B2,point49_B2,point5_B2,point50_B2,point51_B2,point52_B2,point53_B2,point54_B2,point55_B2,point56_B2,point57_B2,point58_B2,point59_B2,point6_B2,point60_B2,point61_B2,point62_B2,point63_B2,point64_B2,point65_B2,point66_B2,point7_B2,point8_B2,point9_B2,point1_B3,point10_B3,point11_B3,point12_B3,point13_B3,point14_B3,point15_B3,point16_B3,point17_B3,point18_B3,point19_B3,point2_B3,point20_B3,point21_B3,point22_B3,point23_B3,point24_B3,point25_B3,point26_B3,point27_B3,point28_B3,point29_B3,point3_B3,point30_B3,point31_B3,point32_B3,point33_B3,point34_B3,point35_B3,point36_B3,point37_B3,point38_B3,point39_B3,point4_B3,point40_B3,point41_B3,point42_B3,point43_B3,point44_B3,point45_B3,point46_B3,point47_B3,point48_B3,point49_B3,point5_B3,point50_B3,point51_B3,point52_B3,point53_B3,point54_B3,point55_B3,point56_B3,point57_B3,point58_B3,point59_B3,point6_B3,point60_B3,point61_B3,point62_B3,point63_B3,point64_B3,point65_B3,point66_B3,point7_B3,point8_B3,point9_B3,point1_B4,point10_B4,point11_B4,point12_B4,point13_B4,point14_B4,point15_B4,point16_B4,point17_B4,point18_B4,point19_B4,point2_B4,point20_B4,point21_B4,point22_B4,point23_B4,point24_B4,point25_B4,point26_B4,point27_B4,point28_B4,point29_B4,point3_B4,point30_B4,point31_B4,point32_B4,point33_B4,point34_B4,point35_B4,point36_B4,point37_B4,point38_B4,point39_B4,point4_B4,point40_B4,point41_B4,point42_B4,point43_B4,point44_B4,point45_B4,point46_B4,point47_B4,point48_B4,point49_B4,point5_B4,point50_B4,point51_B4,point52_B4,point53_B4,point54_B4,point55_B4,point56_B4,point57_B4,point58_B4,point59_B4,point6_B4,point60_B4,point61_B4,point62_B4,point63_B4,point64_B4,point65_B4,point66_B4,point7_B4,point8_B4,point9_B4,point1_B8,point10_B8,point11_B8,point12_B8,point13_B8,point14_B8,point15_B8,point16_B8,point17_B8,point18_B8,point19_B8,point2_B8,point20_B8,point21_B8,point22_B8,point23_B8,point24_B8,point25_B8,point26_B8,point27_B8,point28_B8,point29_B8,point3_B8,point30_B8,point31_B8,point32_B8,point33_B8,point34_B8,point35_B8,point36_B8,point37_B8,point38_B8,point39_B8,point4_B8,point40_B8,point41_B8,point42_B8,point43_B8,point44_B8,point45_B8,point46_B8,point47_B8,point48_B8,point49_B8,point5_B8,point50_B8,point51_B8,point52_B8,point53_B8,point54_B8,point55_B8,point56_B8,point57_B8,point58_B8,point59_B8,point6_B8,point60_B8,point61_B8,point62_B8,point63_B8,point64_B8,point65_B8,point66_B8,point7_B8,point8_B8,point9_B8,point1_B8A,point10_B8A,point11_B8A,point12_B8A,point13_B8A,point14_B8A,point15_B8A,point16_B8A,point17_B8A,point18_B8A,point19_B8A,point2_B8A,point20_B8A,point21_B8A,point22_B8A,point23_B8A,point24_B8A,point25_B8A,point26_B8A,point27_B8A,point28_B8A,point29_B8A,point3_B8A,point30_B8A,point31_B8A,point32_B8A,point33_B8A,point34_B8A,point35_B8A,point36_B8A,point37_B8A,point38_B8A,point39_B8A,point4_B8A,point40_B8A,point41_B8A,point42_B8A,point43_B8A,point44_B8A,point45_B8A,point46_B8A,point47_B8A,point48_B8A,point49_B8A,point5_B8A,point50_B8A,point51_B8A,point52_B8A,point53_B8A,point54_B8A,point55_B8A,point56_B8A,point57_B8A,point58_B8A,point59_B8A,point6_B8A,point60_B8A,point61_B8A,point62_B8A,point63_B8A,point64_B8A,point65_B8A,point66_B8A,point7_B8A,point8_B8A,point9_B8A,point1_B11,point10_B11,point11_B11,point12_B11,point13_B11,point14_B11,point15_B11,point16_B11,point17_B11,point18_B11,point19_B11,point2_B11,point20_B11,point21_B11,point22_B11,point23_B11,point24_B11,point25_B11,point26_B11,point27_B11,point28_B11,point29_B11,point3_B11,point30_B11,point31_B11,point32_B11,point33_B11,point34